In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "3호모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'3호모델_Q_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
# 상장사만 분리한다
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
drop_togehter = tools.dict_data_drop(concat_together7.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 14.20it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 56,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=56)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 62,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=62)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 46,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=46)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 64,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=64)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 91,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=91)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 64,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=64)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 49,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=49)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 39,
  'model': KMeans(max_iter=

## 모델기록


In [18]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

798.0
463.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,463.0,590.0,503.0,506.0,492.0,501.0,514.0,588.0,535.0,580.0,...,731.0,739.0,765.0,700.0,686.0,657.0,562.0,572.0,578.0,581.0
1,524.0,504.0,504.0,525.0,612.0,593.0,501.0,481.0,494.0,502.0,...,526.0,551.0,689.0,581.0,615.0,667.0,729.0,731.0,724.0,798.0
2,607.0,510.0,600.0,577.0,501.0,511.0,585.0,547.0,589.0,536.0,...,719.0,709.0,560.0,744.0,742.0,732.0,764.0,753.0,745.0,660.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,...,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [19]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [20]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [21]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [22]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.884117956632291
1.0


0.1110958077239177

In [23]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

매출총이익                                     0.161410
총자본                                       0.139017
영업이익                                      0.128237
매출채권                                      0.104073
당기순이익                                     0.103140
판매 및 관리비                                  0.101801
영업현금흐름                                    0.083602
최대주주비율                                    0.071700
BW금액+CB금액+유상증자금액                          0.025212
결산기간_주가변동률                                0.020175
차입금의존도                                    0.019613
경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트            0.015448
최대주주비율_변동성                                0.009364
결산기간_거래량증가                                0.006697
최대주주변경카운트                                 0.005694
최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액    0.001992
타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액           0.001709
상호변경카운트+업종변경카운트                           0.001117
dtype: float64

In [24]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.089286,0.05,0.861111,0.085714,0.060606,0.868421,0.047619,0.139535,0.872340,0.139535,...,0.848485,0.011765,0.862069,0.011765,0.923077,0.915493,0.892308,0.102564,0.133333,0.00
1,0.767857,0.85,0.083333,0.914286,0.090909,0.105263,0.857143,0.046512,0.021277,0.023256,...,0.010101,0.117647,0.011494,0.929412,0.025641,0.042254,0.076923,0.051282,0.033333,0.25
2,0.142857,0.10,0.055556,0.000000,0.848485,0.026316,0.095238,0.813953,0.106383,0.837209,...,0.141414,0.870588,0.126437,0.058824,0.051282,0.042254,0.030769,0.846154,0.833333,0.75


In [25]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.089286,0.05,0.861111,0.085714,0.060606,0.868421,0.047619,0.139535,0.872340,0.139535,...,0.848485,0.011765,0.862069,0.011765,0.923077,0.915493,0.892308,0.102564,0.133333,0.00
1,0.767857,0.85,0.083333,0.914286,0.090909,0.105263,0.857143,0.046512,0.021277,0.023256,...,0.010101,0.117647,0.011494,0.929412,0.025641,0.042254,0.076923,0.051282,0.033333,0.25
2,0.142857,0.10,0.055556,0.000000,0.848485,0.026316,0.095238,0.813953,0.106383,0.837209,...,0.141414,0.870588,0.126437,0.058824,0.051282,0.042254,0.030769,0.846154,0.833333,0.75


In [26]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [27]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.787405,-0.053265,0.261189,1,2.5
A000040,-0.277060,0.314095,-0.395487,2,2.5
A000050,0.772184,-0.037709,0.112233,1,2.5
A000070,1.222227,0.281327,0.062858,1,2.5
A000080,1.154792,0.195821,0.160231,1,2.5
...,...,...,...,...,...
A353810,0.058832,0.013551,0.023878,0,2.5
A354200,-0.951799,-0.139476,0.233453,2,2.5
A357780,0.923967,-0.349445,-0.435203,1,2.5
A363280,0.498528,-0.031176,-0.315635,1,2.5
